In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008, 'o3': 'atmpro'}
band = [7]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`          
            layer        coolrg                layer    coolrg
pressure                                                      
0.000312        1 -2.846650e-02                    1 -0.016385
0.000750        2 -2.990319e-02                    2 -0.016788
0.001052        3 -2.966400e-02                    3 -0.015566
0.001476        4 -2.475397e-02                    4 -0.010934
0.002070        5 -1.460090e-02                    5 -0.002472
0.002904        6  1.501729e-03                    6  0.010335
0.004074        7  2.433389e-02                    7  0.028090
0.005714        8  5.472745e-02                    8  0.051542
0.008015        9  9.351221e-02                    9  0.081304
0.011243       10  1.413957e-01                   10  0.118115
0.015771       11  1.994835e-01                   11  0.162972
0.022122       12  2.684707e-01                   12  0.216602
0.031031       13  3.488684e-01                   13  0.279685
0.043528       14  4.696562e-01                   14  0.376439
0.061057       15  6.485656e-01                   15  0.523645
0.085645       16  8.547777e-01                   16  0.699953
0.120136       17  1.073414e+00                   17  0.896803
0.168516       18  1.297531e+00                   18  1.112246
0.236378       19  1.519832e+00                   19  1.341948
0.331549       20  1.736771e+00                   20  1.581007
0.465100       21  1.940360e+00                   21  1.810415
0.652400       22  1.985788e+00                   22  1.912729
0.915100       23  1.671700e+00                   23  1.681419
1.283650       24  1.257081e+00                   24  1.288119
1.800600       25  9.310219e-01                   25  0.940214
2.525700       26  6.512808e-01                   26  0.654089
3.542800       27  4.274313e-01                   27  0.437644
4.969550       28  2.856867e-01                   28  0.300680
6.970850       29  2.114884e-01                   29  0.232614
9.778100       30  1.585146e-01                   30  0.179604
13.715850      31  1.127332e-01                   31  0.129361
19.239350      32  6.586282e-02                   32  0.076805
26.987250      33  3.679478e-02                   33  0.042808
37.855300      34  2.372968e-02                   34  0.025175
53.100050      35 -5.462349e-07                   35 -0.001457
73.887500      36 -3.023340e-02                   36 -0.035313
97.662500      37 -5.601450e-02                   37 -0.066607
121.437500     38 -7.490372e-02                   38 -0.090294
145.212500     39 -8.976174e-02                   39 -0.108040
168.987500     40 -1.023342e-01                   40 -0.121188
192.762500     41 -1.022967e-01                   41 -0.117216
216.537500     42 -9.260374e-02                   42 -0.101579
240.312500     43 -8.243660e-02                   43 -0.086188
264.087500     44 -6.966942e-02                   44 -0.069393
287.862500     45 -5.619760e-02                   45 -0.053544
311.637500     46 -4.145810e-02                   46 -0.038244
335.412500     47 -2.875530e-02                   47 -0.026019
359.187500     48 -2.095323e-02                   48 -0.018670
382.962500     49 -1.529113e-02                   49 -0.013387
406.737500     50 -1.054140e-02                   50 -0.009015
430.512500     51 -6.567970e-03                   51 -0.005419
454.287500     52 -3.557332e-03                   52 -0.002718
478.062500     53 -1.170242e-03                   53 -0.000542
501.837500     54  1.068083e-03                   54  0.001510
525.612500     55  3.220038e-03                   55  0.003471
549.387500     56  5.300104e-03                   56  0.005376
573.162500     57  7.296669e-03                   57  0.007218
596.937500     58  9.204984e-03                   58  0.009001
620.712500     59  1.102190e-02                   59  0.010720
644.487500     60  1.271994e-02                   60  0.012331
668.262500     61  1.427546e-02              

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -10.907597  0.000000 -10.907597 -10.621426 -5.550846e-08   
0.000624    2     -10.907601  0.000002 -10.907599 -10.621429  1.572834e-06   
0.000876    3     -10.907603  0.000003 -10.907600 -10.621431  2.365864e-06   
0.001229    4     -10.907607  0.000006 -10.907601 -10.621433  3.921637e-06   
0.001723    5     -10.907612  0.000010 -10.907602 -10.621436  6.845513e-06   
0.002417    6     -10.907621  0.000017 -10.907604 -10.621442  1.216639e-05   
0.003391    7     -10.907634  0.000030 -10.907603 -10.621450  2.162665e-05   
0.004757    8     -10.907653  0.000053 -10.907600 -10.621462  3.813871e-05   
0.006672    9     -10.907680  0.000093 -10.907587 -10.621479  6.655014e-05   
0.009359    10    -10.907716  0.000159 -10.907557 -10.621501  1.148614e-04   
0.013128    11    -10.907765  0.000271 -10.907494 -10.621530  1.961699e-04   
0.018415    12    -10.907826  0.000456 -10.907369 -10.621564  3.319698e-04   
0.025830    13    -10.907895  0.000762 -10.907133 -10.621598  5.569704e-04   
0.036232    14    -10.907965  0.001261 -10.906704 -10.621624  9.269746e-04   
0.050823    15    -10.908022  0.002130 -10.905892 -10.621623  1.577036e-03   
0.071291    16    -10.908044  0.003724 -10.904319 -10.621561  2.784961e-03   
0.100000    17    -10.907961  0.006548 -10.901412 -10.621358  4.962804e-03   
0.140271    18    -10.907612  0.011320 -10.896292 -10.620850  8.732933e-03   
0.196760    19    -10.906684  0.019074 -10.887609 -10.619724  1.505029e-02   
0.275997    20    -10.904568  0.031224 -10.873344 -10.617399  2.532259e-02   
0.387100    21    -10.900081  0.049595 -10.850486 -10.612784  4.151718e-02   
0.543100    22    -10.890913  0.076283 -10.814630 -10.603905  6.609731e-02   
0.761700    23    -10.875282  0.112074 -10.763208 -10.589168  1.008943e-01   
1.068500    24    -10.854802  0.152347 -10.702454 -10.569722  1.425625e-01   
1.498800    25    -10.830898  0.192519 -10.638378 -10.546654  1.851585e-01   
2.102400    26    -10.803777  0.231967 -10.571810 -10.520041  2.257785e-01   
2.949000    27    -10.776363  0.269868 -10.506495 -10.492827  2.641676e-01   
4.136600    28    -10.753812  0.307447 -10.446365 -10.470049  3.029633e-01   
5.802500    29    -10.739757  0.349769 -10.389988 -10.455284  3.475405e-01   
8.139200    30    -10.733427  0.401979 -10.331448 -10.447803  4.044534e-01   
11.417000   31    -10.735705  0.465805 -10.269900 -10.448637  4.750312e-01   
16.014700   32    -10.754175  0.545673 -10.208502 -10.465601  5.624561e-01   
22.464000   33    -10.807471  0.649285 -10.158185 -10.517533  6.730706e-01   
31.510500   34    -10.913767  0.795012 -10.118755 -10.623530  8.249464e-01   
44.200100   35    -11.079916  0.996830 -10.083085 -10.792354  1.031617e+00   
62.000000   36    -11.324944  1.241857 -10.083087 -11.048344  1.284534e+00   
85.775000   37    -11.668003  1.499769 -10.168233 -11.420461  1.557188e+00   
109.550000  38    -12.011874  1.685886 -10.325988 -11.808821  1.757944e+00   
133.325000  39    -12.348775  1.811834 -10.536941 -12.201975  1.896773e+00   
157.100000  40    -12.690721  1.900983 -10.789738 -12.606541  1.997032e+00   
180.875000  41    -13.047905  1.969961 -11.077944 -13.026468  2.075618e+00   
204.650000  42    -13.386958  2.020914 -11.366044 -13.415274  2.134273e+00   
228.425000  43    -13.684610  2.057764 -11.626846 -13.744181  2.177072e+00   
252.200000  44    -13.944601  2.085588 -11.859013 -14.019482  2.209615e+00   
275.975000  45    -14.161739  2.106514 -12.055225 -14.239518  2.234197e+00   
299.750000  46    -14.336360  2.122866 -12.213495 -14.409372  2.253238e+00   
323.525000  47    -14.468910  2.138656 -12.330254 -14.534297  2.270444e+00   
347.300000  48    -14.568030  2.156792 -12.411238 -14.625828  2.288689e+00   
371.075000  49    -14.648548  2.178299 -12.4702

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')